BASC0038 Algorithms, Logic and Structure

# Week 5: Heapsort

Author: Sam J. Griffiths (sam.griffiths.19@ucl.ac.uk)

---

# Sorting by selection

We can categorise sorting algorithms by their overall strategy. For example, bubble sort (and its variations such as cocktail shaker sort, combsort etc.) can be categorised as sorting by exchange, as the fundamental technique is in repeatedly exchanging elements. Quicksort would also be classed as sorting by exchange for similar reasoning (hence why it sometimes referred to as *partition-exchange sort*).

Insertion sort and variations such as shellsort can be classed as sorting by insertion. Mergesort and its variations can be classed as sorting by merging.

Recall selection sort, so named as it sorts by selection: a sorted sublist is built up by selecting the next element in order one-by-one. In terms of complexity, there is $O(n)$ operations due to selecting elements one-by-one, and for each of those there is an extra $O(n)$ operations in order to determine the next element to select, as it simply compares all remaining elements, giving $O(n^2)$ overall.

What if we could determine the largest/smallest of $n$ elements more efficiently than $O(n)$? Substituting this method into selection sort could then result in an overall compexity better than $O(n^2)$. As it happens, we can indeed perform selection more efficiently than $O(n)$; one way of doing this effectively is with a *heap* data structure.

# Heaps

## Min/max heap property

A *heap* is a type of partially-ordered tree. That is, it a tree adhering to the *heap property*: in a *max heap*, the value at every node is greater than or equal to any of its children. A *min heap* is similar but inverted; that is, the value at every node is less than or equal to any of its children.

A *binary heap* is, as the name suggests, a binary tree adhering to the heap property, i.e. it is a heap where each node has no more than two children.

An example of a max heap might look like:

<img src="https://drive.google.com/uc?export=view&id=1OF322qW3YtnooH2pFmTyOZk4nr6fjSU9" alt="heap.png" width="30%"/>

Note that the heap property applies recursively; that is, any subtree is itself a heap. This also means that the root value in a min/max heap is certain to be the smallest/largest value in the collection.

## Array-based implementation

A binary tree can be represented in practice as an array, i.e. a contiguous sequence of values in memory.

We have seen that in Python (and most programming languages) arrays are zero-indexed. Therefore, in an array representing a binary tree, for any node with index $i$, its relatives are

*   Left child: $2i+1$
*   Right child: $2i+2$
*   Parent: $\lfloor{(i-1)/2}\rfloor$

In this case, the above example of a heap would be represented like so:

In [ ]:
heap = [15, 5, 11, 3, 4, 8, 7]

Note that this representation is simpler than it may appear: it is the *breadth-first order* (or *level-order*) of the tree, that is, it is as though each level of the tree is placed one after another.

## Inserting (pushing) by sift-up

Inserting an element into a heap (commonly referred to as the *push* operation for data structures) is a two stage process. First, the element to push is merely appended to the array, which is equivalent to adding it in the next unfilled position as a child on the bottom level of the binary tree. Then, the heap must be re-balanced to ensure the heap property is maintained by swapping elements. This latter step is referred to as the *sift-up* operation.

Assume we are considering max heaps for now and for the remainder of the worksheet, unless otherwise specified.

Sifting-up a given node is performed by comparing it to its parent and swapping them if it is greater than it. This is performed repeatedly until the node is no longer greater than its parent, or it is now the root of the tree, i.e. it does not have a parent.

We can see that a node on the bottom level of a binary tree with $n$ nodes has $\lfloor{\log{n}}\rfloor$ ancestors. Therefore, the sift-up operation performs $T(n)=\lfloor{\log{n}}\rfloor=O(\log{n})$ comparisons and swaps in the worst case.

In the best case, the inserted value should remain a leaf node (i.e. a node with no children) so no swapping is needed and insertion is constant-time. In the average case, you might expect insertion to be logarithmic-time as with the worst case, but in fact it is also constant-time. The qualitative reasoning for this is that half of the nodes in the tree are leaves, so in the average case the probability that an inserted node should also go on the bottom level is $1/2$. Then, the probability that it should go on the bottom-but-one level is $1/4$, and so on and so forth, which is a converging series and can be used to derive constant-time bounding overall.

## ✍️ Exercise: Sift-up

Implement the sift-up heap operation as `sift_up(heap, start, end)`, where `heap` is a list containing a max heap tree and `start` and `end` form a range within the list containing the heap.

In other words, the root of the heap is at `start` and the final node, to be sifted up, is at `end-1`. As can be seen in the test output, for a whole list simply representing a heap, `start` and `end` are merely `0` and `len(heap)`, respectively. Later, we will see why providing these parameters and thus allowing us to sift on a smaller range within the list becomes useful.

<h2>👇</h2>

In [ ]:
def sift_up(heap, start, end):
  """Sift-up the last node (end-1) in the given max heap.

  Args:
    heap: List containing heap.
    start: Start of range of heap in list.
    end: End of range of heap in list.

  """
  # Swap last node with parents until no longer greater.
  i = end - 1
  heaped = False
  while i > start and not heaped:
    parent = (i - 1) // 2
    if heap[i] > heap[parent]:
      heap[i], heap[parent] = heap[parent], heap[i]
      i = parent
    else:
      heaped = True

🟢

In [ ]:
# Output should be:
# [15, 5, 11, 3, 4, 8]

def push_heap(heap, value):
  heap.append(value)
  sift_up(heap, 0, len(heap))


heap = [11, 5, 8, 3, 4]
push_heap(heap, 15)
print(heap)

[15, 5, 11, 3, 4, 8]


## Extracting (popping) by sift-down

Extracting, a.k.a. *popping*, the root element from the heap &ndash; and thus the smallest/largest element in a min/max heap &ndash; is achieved similarly.

First, the root element is physically removed from the list. As we will consider in future, removing the last element from a list is constant-time, but removing an arbitrary element is linear-time, as all of the following elements must be shifted up in order to fill the resultant gap. If the order does not need to be preserved, however, then any element can be removed from an array via *swap-and-pop*: swap it with the last element, then remove the last element in constant-time.

Therefore, we remove the root element by swapping it with the final element and removing. Then, we must *sift-down* the new root element into its correct position to recover the heap property. This is performed by comparing it to its children and, if it is not larger than or equal to all of them, swapping it with the larger of them. Equivalently to sifting-up, this is repeated until it is larger than or equal to all of its children, or it is on the bottom layer of the tree, i.e. it has no children.

Using the same reasoning as for sifting-up, sifting-down performs $T(n)=2\lfloor{\log{n}}\rfloor=O(\log{n})$ comparisons and $T(n)=\lfloor{\log{n}}\rfloor=O(\log{n})$ swaps in the worst case. 

Unlike insertion, it is also logarithmic-time in the average case. This uses similar reasoning that half of the nodes are on the bottom level, a quarter on the bottom-but-one etc. Whereas insertion starting from the bottom meant that, on average, nodes would not require sifting up to upper levels, the same reasoning means that starting from the top, most nodes will need sifting down to lower levels, in logarithmic-time. It is similarly trivially constant-time in the best case.

## ✍️ Exercise: Sift-down

Implement the sift-down heap operation as `sift_down(heap, start, end)`, where `heap` is a list containing a max heap tree and `start` and `end` form a range within the list containing the heap.

In other words, the root of the heap, to be sifted down, is at `start` and the final node is at `end-1`. As can be seen in the test output, for a whole list simply representing a heap, `start` and `end` are merely `0` and `len(heap)`, respectively.

<h2>👇</h2>

In [ ]:
def sift_down(heap, start, end):
  """Sift-down the first node (start) in the given max heap.

  Args:
    heap: List containing heap.
    start: Start of range of heap in list.
    end: End of range of heap in list.

  """
  # Swap first node with children until no longer smaller.
  i = start
  heaped = False
  while not heaped:
    left = i * 2 + 1
    right = i * 2 + 2
    largest = i

    # Find largest of i, left and right
    if left < end and heap[left] > heap[largest]:
      largest = left
    if right < end and heap[right] > heap[largest]:
      largest = right

    # If left or right is larger than i, swap and repeat
    if largest == i:
      heaped = True
    else:
      heap[i], heap[largest] = heap[largest], heap[i]
      i = largest

🟢

In [ ]:
# Output should be:
# 15
# [11, 5, 8, 3, 4]

def pop_heap(heap):
  heap[0], heap[-1] = heap[-1], heap[0]
  value = heap.pop()
  sift_down(heap, 0, len(heap))
  return value


heap = [15, 5, 11, 3, 4, 8]
max = pop_heap(heap)
print(max)
print(heap)

15
[11, 5, 8, 3, 4]


# Heapsort

We have seen that a heap can be used to select and remove the smallest/largest element from a collection in logarithmic-time, a clear improvement over a naive linear-time search.

*Heapsort* is a sorting algorithm which sorts by selection by inserting all of the elements into a heap and then repeatedly extracts them to form a sorted list. Using the functions implemented so far, a simple, naive implementation of heapsort from this description could look like this: 

In [ ]:
# Output should be:
# [-2, 0, 4, 5, 6, 14, 21, 23, 28, 30, 31, 32, 38, 38, 39, 42, 42, 48, 49, 52,
# 52, 53, 56, 69, 75, 83, 83, 88, 95]

def heapsort_naive(array):
  heap = []
  for value in array:
    push_heap(heap, value)

  result = []
  while len(heap) > 0:
    result.append(pop_heap(heap))

  result.reverse()
  return result


data = [48, 38, 28, 14, 38, 56, 49, 30, 39, 0, 5, 4, 31, 88, 83, 32, 52, 42,
        83, -2, 69, 95, 75, 52, 21, 23, 53, 42, 6]
sorted_data = heapsort_naive(data)
print(sorted_data)

[-2, 0, 4, 5, 6, 14, 21, 23, 28, 30, 31, 32, 38, 38, 39, 42, 42, 48, 49, 52, 52, 53, 56, 69, 75, 83, 83, 88, 95]


Note that this is an out-of-place, functional-style implementation. As we have implemented a max heap, the result is sorted from largest to smallest, so it must be reversed for a result sorted from smallest to largest. This would be unnecessary if a min heap was used, instead.

The complexity analysis is equivalent to pushing $n$ elements into a heap and then popping $n$ elements (ignoring the reversal). In the worst case, this is $T(n) \approx nO(\log{n})+nO(\log{n}) = O(n\log{n}).$ In the average case, this is $T(n) \approx nO(1)+nO(\log{n}) = O(n\log{n})$.

As clarified in 4-quicksort, it is generally more beneficial in terms of memory usage to perform an in-place sort, rearranging the given data rather than providing a sorted copy. A more sophisticated, in-place version of heapsort is as follows:

1.   Instead of initialising an empty heap and pushing each element into it, treat the array `start` to `end` as given as a broken max heap and restore the heap property.
2.   Swap the first element of the array (and thus the heap root) with the element. This is now in the final, sorted position. Sift down the new root element to fix the heap `start` to `end-1`.
3.   Repeat by swapping the new first element into the second-to-last position. The sorted sublist builds up from the end. Sift down the new root element to fix the heap `start` to `end-2`.
4.   Repeat until the heap is reduced to only the first element, which is necessarily the smallest. Thus, the array is sorted.

In point 1, an arbitrary array of values is 'heapified' by calling sift-down on every non-leaf node from the bottom-up. This works by considering subtrees. That is, the subtrees rooted with every leaf are already heaps, as they are just single nodes. Then, sifting down each node on the next level up turns each of the subtrees rooted in those nodes into a heap. This is repeated moving up level by level until, finally, the root value itself is sifted down to finish heapifying the entire tree.

So, the array is heapified in-place by repeated sift-down operations, then the root is continually popped into a sorted sublist in the back of the array. In terms of complexity, consider again that an individual insertion by sift-up is $O(\log{n})$ in the worst case and $O(1)$ in the average case, such that building a heap by one-by-one insertion is $O(n\log{n})$ in the worst case and $O(n)$ in the average case. By instead heapifying in-place by repeated sift-downs, this is improved to $O(n)$ *in the worst case*. The intuitive reasoning for this is, again, similar to that dicussed in the analysis of sift-up and sift-down: sifting-down a node naturally takes fewer operations the lower down that node is, and most of the nodes in the tree are found in the lower levels.

Therefore, both the worst- and average-case complexities of heapsort are of the form

\begin{align}
T(n) &\approx O(n)+nO(\log{n}) \\
&= O(n\log{n})
\end{align}

# ✍️ Exercise: In-place heapsort

First, implement an in-place heapification as `heapify(array)`, which reorders the given `array` into a max heap via repeated calls to your `sift_down` function as discussed above.

Secondly, implement the final heapsort algorithm as `heapsort(array)`, which sorts the given `array` by heapifying in-place, then popping root elements and swapping them into the back of the array, also as discussed above.

<h2>👇</h2>

In [ ]:
def heapify(array):
  """Reorder a given array into a max heap.

  Args:
    array: List to heapify.

  """
  # Start by sifting down the first parent node
  n = len(array)
  node = (n - 2) // 2

  # Sift down all nodes, finishing with the root
  while node >= 0:
    sift_down(array, node, n)
    node -= 1


def heapsort(array):
  """Sort a list in-place via heapsort.

  Args:
    array: Unsorted list.

  """
  # Turn the entire array into a heap
  heapify(array)

  # Repeatedly extract the root from the heap into a sorted sublist
  n = len(array)
  while n > 1:
    array[0], array[n - 1] = array[n - 1], array[0]
    n -= 1
    sift_down(array, 0, n)

🟢

In [ ]:
# Output should be:
# [-2, 0, 4, 5, 6, 14, 21, 23, 28, 30, 31, 32, 38, 38, 39, 42, 42, 48, 49, 52,
# 52, 53, 56, 69, 75, 83, 83, 88, 95]

data = [48, 38, 28, 14, 38, 56, 49, 30, 39, 0, 5, 4, 31, 88, 83, 32, 52, 42,
        83, -2, 69, 95, 75, 52, 21, 23, 53, 42, 6]
heapsort(data)
print(data)

[-2, 0, 4, 5, 6, 14, 21, 23, 28, 30, 31, 32, 38, 38, 39, 42, 42, 48, 49, 52, 52, 53, 56, 69, 75, 83, 83, 88, 95]


# ➕ Extra: Timing experiments

Repeat some timing experiments in the same vein as those previously to confirm some of the many claims made in this worksheet! For example, try to produce some graphs showing:

*   Logarithmic-time worst-case sift-up vs constant-time average-case.
*   Logarithmic-time worst-case sift-down vs logarithmic-time average-case.
*   Linearithmic-time worst-case heap building (by repeated insertions) vs linear-time average-case.
*   Linear-time worst-case heapification (by repeated sift-down) vs linear-time average-case.
*   Linearithmic-time worst-case heapsort vs linearithmic-time average-case.

